In [21]:
# Imports for the data to be processed
import pandas as pd
import numpy as np
import seaborn as sns
from copy import copy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
import plotly.express as px
import matplotlib.ticker as ticker
from matplotlib import pyplot
from sklearn.model_selection import StratifiedKFold
import re


In [9]:
NotebookApp.iopub_data_rate_limit=1.0e10 

NameError: name 'NotebookApp' is not defined

In [22]:

df_all_immigrant_data = pd.read_csv('immigration_statistics.csv', encoding='cp1252')
new_header = df_all_immigrant_data.iloc[1] #grab the first row for the header
df_all_immigrant_data = df_all_immigrant_data[2:] #take the data less the header row
df_all_immigrant_data.columns = new_header #set the header row as the df header
df_all_immigrant_data.dropna(axis = 0, inplace = True)

df_all_immigrant_data_accepted = df_all_immigrant_data[df_all_immigrant_data['Admissions'].str.contains(r'\d')]
admissions_count = df_all_immigrant_data_accepted['Admissions'].str.extract(r'(\d+)', expand=False).astype(int)
new_df = df_all_immigrant_data_accepted.loc[df_all_immigrant_data_accepted.index.repeat(admissions_count)].reset_index(drop=True)

valid_rows = df_all_immigrant_data[(df_all_immigrant_data['Admissions'] == ' D ')]
new_df = pd.concat([new_df, valid_rows], ignore_index=True)

valid_rows = df_all_immigrant_data[(df_all_immigrant_data['Admissions'] == 'D')]
immigrant_data = pd.concat([new_df, valid_rows], ignore_index=True)

admission_codes = []
for idx, row in immigrant_data.iterrows():
    if row['Admissions'] == 'D' or row['Admissions'] == ' D ':
        admission_codes.append('No')
    else:
        admission_codes.append('Yes')
immigrant_data['Admission Codes'] = admission_codes
immigrant_data = immigrant_data.drop(['County of Residence'], axis = 1)
immigrant_data['Major Class of Admission'].unique()

array(['Immediate Relatives - Spouses', 'Employment Preference 2nd',
       'Employment Preference 4th', 'Immediate Relatives - Parents',
       'Family Preference 2nd', 'Employment Preference 1st',
       'Immediate Relatives - Children',
       'Employment Preference 3rd Skilled', 'Diversity Program', 'Other',
       'Family Preference 3rd', 'Refugee and Asylee Adjustments',
       'Family Preference 4th', 'Employment Preference 3rd Unskilled',
       'Employment Preference 5th', 'Family Preference 1st'], dtype=object)

In [23]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score


categorical_features = ['State of Residence',  'Country of Birth', 'Major Class of Admission']

# Split data into features and labels
X = immigrant_data.drop(['Admissions', 'Admission Codes'], axis=1)
y = immigrant_data['Admission Codes']

# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False)
x_categorical = encoder.fit_transform(X[categorical_features])
X_categorical = pd.DataFrame(x_categorical, columns=encoder.get_feature_names(categorical_features))
X = pd.concat([X.drop(categorical_features, axis=1), X_categorical], axis=1)

# initialize of kfold object
k_fold = StratifiedKFold(n_splits= 4)
clf = LogisticRegression(solver='liblinear')
accuracy_scores = []

for train_idx, test_idx in k_fold.split(X, y):    
    # split into train and test sets
    x_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    x_test = X.iloc[test_idx, :]
    y_test = y.iloc[test_idx]
    
    # fit classifier (on training set)
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

list_of_user_input = {'State of Residence': [], 'Country of Birth': [], 'Major Class of Admission': []}
for item in categorical_features:
    value_from_user = (input(f'please enter the {item}: '))
    list_of_user_input[item] = [value_from_user]
sample_df = pd.DataFrame(list_of_user_input)  



/Users/mayaprasad/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


please enter the State of Residence: New Jersey
please enter the Country of Birth: France
please enter the Major Class of Admission: Immediate Relatives - Spouses


In [24]:

# Transform user input using the same encoder
sample_info = encoder.transform(sample_df[categorical_features])
sample_info = pd.DataFrame(sample_info, columns=encoder.get_feature_names(categorical_features))

# Reorder columns to match the trained classifier
sample_df = pd.concat([sample_df.drop(categorical_features, axis=1), sample_info], axis=1)
sample_df = sample_df.reindex(columns=X.columns, fill_value=0)  # Add missing columns with 0 values

# Make prediction for person data
prediction = clf.predict(sample_df)
print('Prediction:', prediction)

Prediction: ['Yes']


/Users/mayaprasad/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [26]:
# Calculate average accuracy
average_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print('Average Accuracy:', average_accuracy)

Average Accuracy: 0.8782618987941071


In [27]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest, f_classif
from scipy import sparse
import joblib

def evaluate_fold(x_train, y_train, x_test, y_test, classifier):
    # Fit classifier (on training set)
    classifier.fit(x_train, y_train)
    y_pred = classifier.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

categorical_features = ['State of Residence', 'Country of Birth', 'Major Class of Admission']

# Split data into features and labels
X = immigrant_data.drop(['Admissions', 'Admission Codes'], axis=1)
y = immigrant_data['Admission Codes']

# One-hot encode categorical features
encoder = OneHotEncoder(sparse=False)
x_categorical = encoder.fit_transform(X[categorical_features])
X_categorical = pd.DataFrame(x_categorical, columns=encoder.get_feature_names(categorical_features))
X = pd.concat([X.drop(categorical_features, axis=1), X_categorical], axis=1)

# Perform feature selection
selector = SelectKBest(score_func=f_classif, k=10)
X_selected = selector.fit_transform(X, y)
selected_feature_indices = selector.get_support(indices=True)
X = X.iloc[:, selected_feature_indices]

# Initialize the k-fold object
k_fold = StratifiedKFold(n_splits=3)

# Initialize and configure the classifier
clf = LogisticRegression()

# Perform parallel processing for cross-validation
accuracy_scores = joblib.Parallel(n_jobs=-1)(
    joblib.delayed(evaluate_fold)(X.iloc[train_idx, :], y.iloc[train_idx], X.iloc[test_idx, :], y.iloc[test_idx], clf)
    for train_idx, test_idx in k_fold.split(X, y)
)

mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
print("Mean accuracy:", mean_accuracy)




/Users/mayaprasad/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Mean accuracy: 0.9235142172835381
